# MLP Bag of Words

En esta notebook, haremos un clasificador en base a una MLP + Bag of Words



In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/test_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


In [2]:
from hate.nn.preprocessing import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tok_args = {
    "preserve_case": False,
    "deaccent": False,
    "reduce_len": True,
    "strip_handles": False,
    "alpha_only": False,
    "stem": True
}

tokenizer = Tokenizer(**tok_args)


vect = CountVectorizer(tokenizer=tokenizer.tokenize, 
                       max_df=0.65, min_df=0.00075, ngram_range=(1, 2), binary=True)

vect.fit(df_train["text"])

bow_train = vect.transform(df_train["text"])
bow_dev = vect.transform(df_dev["text"])

bow_train.shape, bow_dev.shape

Using TensorFlow backend.


((4500, 5649), (500, 5649))

In [3]:
X_train, y_train = df_train["text"], df_train["HS"]
X_dev, y_dev = df_dev["text"], df_dev["HS"]

X_train.shape, y_train.shape

((4500,), (4500,))

In [4]:
from hate.nn import BowModel
from keras.optimizers import Adam

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01,
}

vect_args = {
    "max_df": 0.65, 
    "min_df": 0.00075,
    "ngram_range": (1, 2),
    "binary": True
}

model = BowModel(num_words=5500, 
    tokenize_args=tok_args, vectorize_args=vect_args,
    dense_units=[512, 128], dropout=[0.75, 0.50]
)

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/10
4500/4500 [==============================] - 8s 2ms/step - loss: 0.6156 - acc: 0.6736 - val_loss: 0.5150 - val_acc: 0.7700
Epoch 2/10
4500/4500 [==============================] - 7s 2ms/step - loss: 0.4395 - acc: 0.8120 - val_loss: 0.4607 - val_acc: 0.7980
Epoch 3/10
4500/4500 [==============================] - 8s 2ms/step - loss: 0.3484 - acc: 0.8638 - val_loss: 0.4451 - val_acc: 0.8220
Epoch 4/10
4500/4500 [==============================] - 7s 2ms/step - loss: 0.2823 - acc: 0.8891 - val_loss: 0.4512 - val_acc: 0.8180
Epoch 5/10
4500/4500 [==============================] - 8s 2ms/step - loss: 0.2391 - acc: 0.9127 - val_loss: 0.4567 - val_acc: 0.8120
Epoch 6/10
4500/4500 [==============================] - 8s 2ms/step - loss: 0.2039 - acc: 0.9269 - val_loss: 0.4662 - val_acc: 0.8180
Epoch 7/10
4500/4500 [==============================] - 7s 2ms/step - loss: 0.1853 - acc: 0.9362 - val_loss: 0.4722 - val_acc: 0.8140
Epoch 8/10
4500

In [5]:
from hate.utils import print_evaluation


print_evaluation(model, X_dev, y_dev)

500/500 [==============================] - 0s 255us/step
Loss        : 0.5059
Accuracy    : 0.8140
Precision   : 0.8116
Recall      : 0.7568
F1          : 0.7832


## Error Analysis

In [6]:
df_dev["proba"] = model.predict(X_dev)


true_positives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] >= 0.5)].copy()
true_negatives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] < 0.5)].copy()

false_positives = df_dev[(df_dev["HS"] == 0) & (df_dev["proba"] > 0.5)].copy()
false_positives.sort_values("proba", ascending=False, inplace=True)


false_negatives = df_dev[(df_dev["HS"] == 1) & (df_dev["proba"] < 0.5)].copy()
false_negatives.sort_values("proba", ascending=True, inplace=True)

conf_matrix = pd.DataFrame([
    {"real":"hs=1", "pred_true": len(true_positives), "pred_false": len(false_negatives)},
    {"real":"hs=0", "pred_true": len(false_positives), "pred_false": len(true_negatives)}
])



conf_matrix.set_index("real", inplace=True)

print("Falsos negativos: {}".format(len(false_negatives)))
print("Falsos positivos: {}".format(len(false_positives)))

conf_matrix[["pred_true", "pred_false"]]

Falsos negativos: 54
Falsos positivos: 39


,pred_true,pred_false
real,,
hs=1,168,54
hs=0,39,239


## Falsos Negativos


In [7]:
pd.set_option('max_colwidth', 300)

false_negatives[["text", "proba"]]


,text,proba
id,,
20711,@lopezdoriga eso es culpa de los gobiernos corruptos que tienen arreglos con esta escoria. Ojalá se muera la perra esa...,0.000759
21152,@NatyBurgos15 @Horaciogenta @PaolaPa05 Cállate vos Black and bitch como la puta falsa abogada. Espero se pudra en la cárcel y devuelva todo. Dedícate a estudiar en serio. La droga es mala. Cristina puta,0.003225
21072,"Voy andando por la calle y me han dicho cosas en árabe ya que me estas acosando dime lo en español para que te entienda, asique como yo no les entiendo me he cagado en su puta madre en árabe para que ellos a mi si.",0.004501
23716,"- El otro día me comí a Laura es muy fácil que guarra jajajaja - Laura también te comió a ti entonces tú eres igual de guarro e igual de fácil. - Puto feminazi putas modas eres feminista solo para ligar bien queda gilipollas - em, ok",0.004718
21215,"Que tal otro aporte anónimo son de la puta de mi esposa, así los dejó en navidad enseñando le los calzones al vecino rt y siguenos @TANGASUCIA @Tangasrobadas https://t.co/70ioUjveqY",0.006560
24925,"La verdad que si, no veia la hora de llegar a los 5 mil amigos😂 ahora de a poco voy borrando esos indios indocumentados😂 https://t.co/Xw8q4GLM8v",0.009737
22065,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,0.011905
22199,Todo aquél que se haya molestado en investigar un poco sabe que desde tiempos antiguos los mayores negreros y esclavistas del mundo fueron los árabes. https://t.co/DY5Uo8979v,0.016115
22574,yo: quién tú eres tú bizcochitooo (cantando) mi mamá: a la tenés clara puta Jajajajaja me estallo,0.016998


¿Cómo los tokenizamos?

# Conclusiones

- Modelo BoW con una MLP anda bastante bien

No estarían convenciendo las traducciones...

Diría que intentemos un ensemble con caracteres o n-gramas.